<a href="https://colab.research.google.com/github/Sivaramjallu001/GSTIN_Certificate_Verification/blob/main/Signature_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install pytesseract and Tesseract OCR engine
!apt-get install -y tesseract-ocr
!pip install pytesseract pillow


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [30]:
import cv2
import pytesseract

# Load image and preprocess
def preprocess(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, None, fx=2, fy=2, interpolation=cv2.INTER_LINEAR)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return img, thresh

# Extract coordinates for keywords
def extract_signature_keywords(thresh_img):
    data = pytesseract.image_to_data(thresh_img, output_type=pytesseract.Output.DICT)
    keywords = ["goods", "digitally", "verified"]
    boxes = {}

    for i, word in enumerate(data['text']):
        word_clean = word.strip().lower()
        for key in keywords:
            if key in word_clean:
                x, y, w, h = data['left'][i], data['top'][i], data['width'][i], data['height'][i]
                boxes[key] = (x, y, w, h)
    return boxes

# Compare bounding box coordinates with reference
def compare_positions(ref_boxes, test_boxes, tolerance=500):
    for key in ref_boxes:
        if key not in test_boxes:
            print(f"❌ Keyword '{key}' not found in test image.")
            return False
        rx, ry, _, _ = ref_boxes[key]
        tx, ty, _, _ = test_boxes[key]
        if abs(rx - tx) > tolerance or abs(ry - ty) > tolerance:
            print(f"❌ Keyword '{key}' misaligned: Ref ({rx},{ry}) vs Test ({tx},{ty})")
            return False
    print("✅ All key signature elements are aligned correctly.")
    return True

# ---------- Run the Comparison ----------
# Reference image: a known valid GSTIN certificate
ref_img_path = "/content/gst_certificate.jpeg"
test_img_path = "/content/WhatsApp Image 2025-05-30 at 15.27.00.jpeg"

ref_img, ref_thresh = preprocess(ref_img_path)
test_img, test_thresh = preprocess(test_img_path)

ref_boxes = extract_signature_keywords(ref_thresh)
test_boxes = extract_signature_keywords(test_thresh)

compare_positions(ref_boxes, test_boxes)


✅ All key signature elements are aligned correctly.


True

In [33]:
from PIL import Image

# Convert to JPEG (one-time)
im = Image.open("/content/2.webp")
im.convert("RGB").save("/content/2_converted.jpg", "JPEG")
test_img_path = "/content/2_converted.jpg"


In [34]:
ref_img_path = "/content/WhatsApp Image 2025-05-30 at 15.27.00.jpeg"

ref_img, ref_thresh = preprocess(ref_img_path)
test_img, test_thresh = preprocess(test_img_path)

ref_boxes = extract_signature_keywords(ref_thresh)
test_boxes = extract_signature_keywords(test_thresh)

compare_positions(ref_boxes, test_boxes)

❌ Keyword 'goods' not found in test image.


False

In [28]:
# Convert to JPEG (one-time)
im = Image.open("/content/1.webp")
im.convert("RGB").save("/content/1_converted.jpg", "JPEG")
test_img_path = "/content/1_converted.jpg"

In [32]:
ref_img_path = "/content/WhatsApp Image 2025-05-30 at 15.27.00.jpeg"

ref_img, ref_thresh = preprocess(ref_img_path)
test_img, test_thresh = preprocess(test_img_path)

ref_boxes = extract_signature_keywords(ref_thresh)
test_boxes = extract_signature_keywords(test_thresh)

compare_positions(ref_boxes, test_boxes)

✅ All key signature elements are aligned correctly.


True